In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import gradio as gr

# 1. Generate dummy data
def generate_dummy_data():
    x = np.linspace(0, 100, 500)
    y = np.sin(x / 5) + np.random.normal(scale=0.1, size=len(x))
    return y

data = generate_dummy_data()

In [10]:
# 2. Prepare dataset (with time_steps = 5)
def create_dataset(data, time_steps=5):  # 👈 changed from 10 to 5
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_dataset(data)
X = X.reshape((X.shape[0], X.shape[1], 1))


In [11]:
# 3. Build model for input shape (5, 1)
model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(5, 1)),  # 👈 changed from (10, 1) to (5, 1)
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, verbose=0)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
import pickle

with open('/content/drive/MyDrive/Divya/stock.pkl', 'wb') as file:
    pickle.dump(model,file)

In [14]:
with open('/content/drive/MyDrive/Divya/stock.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [15]:
# 4. Predict function
def predict_next_price(seq):
    try:
        seq = [float(i.strip()) for i in seq.split(',')]
        if len(seq) != 5:  # 👈 only 5 numbers expected now
            return "Please enter exactly 5 numbers."

        input_seq = np.array(seq).reshape((1, 5, 1))  # 👈 reshape accordingly
        pred = model.predict(input_seq)
        return f"📈 Predicted next price: {pred[0][0]:.4f}"
    except Exception as e:
        return f"Error: {str(e)}"

# 5. Gradio UI
iface = gr.Interface(
    fn=predict_next_price,
    inputs=gr.Textbox(lines=2, placeholder="Enter 5 stock prices, comma-separated"),
    outputs="text",
    title="📊 Stock Price Predictor (RNN)",
    description="Enter 5 stock prices to predict the next one."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e823cca7b66be05b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
